<a href="https://colab.research.google.com/github/cuddihyd/baruch-mth-9796/blob/nyt-ft-2024/nyt-expt-24/20240228_fine_tune_pretrained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tune Pretrained Model
Following the HF docs:

https://huggingface.co/docs/transformers/en/training#train-with-pytorch-trainer


### SymLinks
```
ln -s drive/MyDrive/baruch-nlp/2024-project-spike/ .
```


### requirements.txt

```
datasets
transformers[torch]
tensorflow>=2.15
evaluate

```

In [ ]:
!pip install -r requirements.txt

In [1]:
import tensorflow
tensorflow.__version__

'2.15.0'

In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("yelp_review_full")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


In [8]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
from transformers import AutoModelForSequenceClassification

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from transformers import TrainingArguments

In [13]:
training_args = TrainingArguments(output_dir="test_trainer")

In [14]:
import numpy as np

In [15]:

import evaluate


In [16]:
metric = evaluate.load("accuracy")


In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer


In [19]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.651252,0.174000
2,No log,1.090242,0.515000
3,No log,1.010948,0.565000


TrainOutput(global_step=375, training_loss=1.3507430826822917, metrics={'train_runtime': 2674.8258, 'train_samples_per_second': 1.122, 'train_steps_per_second': 0.14, 'total_flos': 789354427392000.0, 'train_loss': 1.3507430826822917, 'epoch': 3.0})

...


For first round: ETA=45 minutes to run the training... (actually finished in 42m)

...Then it keeps going. Not sure how many Epochs it will do. To be researched how it decides if/when to stop.
Ah. OK Just stopped.



In [22]:
trainer.save_model('drive/MyDrive/baruch-nlp/2024-project-spike/20240228/yelp-trained-classifier.model')
